# makemore-jax

In [102]:
import random

import numpy as np
import jax.numpy as jnp
import jax.random as jrandom

from jax import grad, value_and_grad

## Data

In [19]:
with open('names.txt') as f:
    words = f.read().splitlines()

len(words)

32033

In [20]:
chars = sorted(set(''.join(words)))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}

len(stoi)

27

In [22]:
def build_dataset(words, block_size=3):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    return np.array(X), np.array(Y)

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

len(Ytr), len(Ydev), len(Yte)

(182580, 22767, 22799)

## Model

In [185]:
d = 2
neurons = 100

key = jrandom.PRNGKey(0)
key, *subkeys = jrandom.split(key, 6)
subkeys = iter(subkeys)

C = jrandom.normal(next(subkeys), shape=(27, d))
W1 = jrandom.normal(next(subkeys), shape=(d * 3, neurons))
b1 = jrandom.normal(next(subkeys), shape=(neurons, ))
W2 = jrandom.normal(next(subkeys), shape=(neurons, 27))
b2 = jrandom.normal(next(subkeys), shape=(27, ))

parameters = [C, W1, b1, W2, b2]
sum(p.size for p in parameters)

3481

## Training

In [186]:
def loss_fn(C, W1, b1, W2, b2, X, Y):
    emb = C[X]
    h = jnp.tanh(emb.reshape((-1, 6)) @ W1 + b1)
    logits = h @ W2 + b2
    logits -= logits.max(1, keepdims=True)
    counts = jnp.exp(logits)
    probs = counts / counts.sum(1, keepdims=True)
    loss = -jnp.log(probs[jnp.arange(X.shape[0]), Y]).mean()
    return loss

In [187]:
key = jrandom.PRNGKey(0)
lr = 0.1
for _ in range(10000):
    key, subkey = jrandom.split(key)
    ix = jrandom.randint(subkey, shape=(32,), minval=0, maxval=Xtr.shape[0])
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    loss, (gC, gW1, gb1, gW2, gb2) = value_and_grad(loss_fn, argnums=(0, 1, 2, 3, 4))(C, W1, b1, W2, b2, Xb, Yb)

    C -= lr * gC
    W1 -= lr * gW1
    b1 -= lr * gb1
    W2 -= lr * gW2
    b2 -= lr * gb2

In [188]:
loss_fn(C, W1, b1, W2, b2, Xtr, Ytr), loss_fn(C, W1, b1, W2, b2, Xdev, Ydev)

(Array(2.4897997, dtype=float32), Array(2.4876149, dtype=float32))

## Visualization

In [189]:
import matplotlib.pyplot as plt
plt.scatter(C[:, 0], C[:, 1], s=200)
for i in range(C.shape[0]):
    plt.text(C[i, 0], c[i, 1], itos[i], ha="center", va="center", color="white")

ModuleNotFoundError: No module named 'matplotlib'